In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression

from tensorflow import keras
from keras import layers
from keras.callbacks import Callback
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from textblob import TextBlob
from textblob import Word

import re
import string

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_data_file = "drive/My Drive/Final year project/train.csv"
test_data_file = "drive/My Drive/Final year project/test.csv"
submission_file = "drive/My Drive/Final year project/test.csv"

train_data = pd.read_csv(train_data_file)
test_data = pd.read_csv(test_data_file)
submission_result = pd.read_csv(test_data_file)

# Preprocessing

max_len is the maximum length of a sequence of words (in terms of number of tokens) that will be input to a neural network model. Any sequence longer than this will be truncated, and any sequence shorter than this will be padded with zeros.

embedding_dim is the dimensionality of the word embeddings that will be learned by the neural network. Word embeddings are vector representations of words that capture semantic and syntactic relationships between words.

vocabulary_size is the maximum number of unique words that the model will consider. Any words outside of this vocabulary will be treated as out-of-vocabulary (OOV) words.

num_tokens is equal to the vocabulary size plus one, since we need to include a token for OOV words. In this case, the OOV token is represented by 0

In [ ]:
# set up paramters
max_len = 120
embedding_dim = 300
vocabulary_size = 20000 #35000
num_tokens = vocabulary_size+1 #including 0

In [ ]:
# preprocess comment texts
def preprocess(corpus):

# remove all non-English characters
# and convert all letters to lower case
    printable = set(string.printable)
    corpus = ''.join(filter(lambda x: x in printable, corpus))
    corpus = corpus.lower()

# change contracted words into possible non-contracted form
    # specific
    corpus = re.sub(r"won't", "will not", corpus)
    corpus = re.sub(r"can\'t", "can not", corpus)
    # or could be 'are not' etc
    corpus = re.sub(r"ain\'t","is not", corpus)
    corpus = re.sub(r"shan\'t", "shall not", corpus)
    corputs = re.sub(r"let\'s", "let us", corpus)

    # general
    corpus = re.sub(r"n\'t", " not", corpus)
    corpus = re.sub(r"\'re", " are", corpus)
    corpus = re.sub(r"\'s", " is", corpus)
    # or could be \'d --> had
    corpus = re.sub(r"\'d", " would", corpus)
    corpus = re.sub(r"\'ll", " will", corpus)
    corpus = re.sub(r"\'t", " not", corpus)
    corpus = re.sub(r"\'ve", " have", corpus)
    corpus = re.sub(r"\'m", " am", corpus)

    # replace the rest \' with ' '
    corpus = re.sub(r"\'", " ", corpus)

    correction_list = {"youfuck": "you fuck", \
                       "fucksex": "fuck sex",\
                       "bitchbot": "bitch bot",\
                       "offfuck": "fuck off",\
                       "donkeysex": "donkey sex",\
                      "securityfuck": "security fuck",\
                      "ancestryfuck": "ancestry fuck",\
                      "turkeyfuck": "turkey fuck",\
                      "faggotgay": "faggot gay",\
                       "fuckbot": "fuck bot",\
                       "assfuckers": "ass fucker",\
                       "ckckck": "cock",\
                       "fuckfuck": "fuck",\
                       "lolol": "lol",\
                       "pussyfuck": "fuck",\
                        "gaygay": "gay",\
                       "haha": "ha",\
                       "sucksuck": "suck"
                      }
    for old,new in correction_list.items():
        corpus = corpus.replace(old,new)

    return corpus

This is a Python function that performs text preprocessing on a given corpus. The function takes a single argument, corpus, which is a string of text. The text preprocessing steps that are performed by this function are described below:

The first step is to remove all non-English characters from the corpus string, and convert all letters to lowercase. This is done using the filter() function and the lower() method of the string class.

The next step is to replace contracted words with their possible non-contracted form. For example, "won't" is replaced with "will not", "can't" is replaced with "can not", and so on. This is done using regular expressions and the re.sub() function.

The third step is to replace other common contractions with their expanded form. For example, "n't" is replaced with "not", "'re" is replaced with "are", "'s" is replaced with "is", and so on. This is also done using regular expressions and the re.sub() function.

Finally, any remaining apostrophes (') are replaced with spaces. This is done using another call to re.sub().

Overall, this function is useful for standardizing and cleaning up text data before further analysis or processing, and can be used as a pre-processing step in natural language processing pipelines.







In [ ]:
# tokenizer vocabulary_size words
# we ignore all numbers
tokenizer = Tokenizer(num_words = vocabulary_size+1,\
                      filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n0123456789',\
                      lower=True, split=' ')

# extract comment texts from train_data and test_data
X_train_raw = train_data["comment_text"]
X_test_raw = test_data["comment_text"]

bad_comment_cat = ['toxic', 'severe_toxic', 'obscene', 'threat',\
       'insult', 'identity_hate']
Y_train = train_data[bad_comment_cat]

This is a Python function that performs text preprocessing on a given corpus. The function takes a single argument, corpus, which is a string of text. The text preprocessing steps that are performed by this function are described below:

The first step is to remove all non-English characters from the corpus string, and convert all letters to lowercase. This is done using the filter() function and the lower() method of the string class.

The next step is to replace contracted words with their possible non-contracted form. For example, "won't" is replaced with "will not", "can't" is replaced with "can not", and so on. This is done using regular expressions and the re.sub() function.

The third step is to replace other common contractions with their expanded form. For example, "n't" is replaced with "not", "'re" is replaced with "are", "'s" is replaced with "is", and so on. This is also done using regular expressions and the re.sub() function.

Finally, any remaining apostrophes (') are replaced with spaces. This is done using another call to re.sub().

Overall, this function is useful for standardizing and cleaning up text data before further analysis or processing, and can be used as a pre-processing step in natural language processing pipelines.




jensy.doshi74@nmims.edu.in
tokenizer = Tokenizer(num_words = vocabulary_size+1,\
                      filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n0123456789',\
                      lower=True, split=' ')

# extract comment texts from train_data and test_data
X_train_raw = train_data["comment_text"]
X_test_raw = test_data["comment_text"]

bad_comment_cat = ['toxic', 'severe_toxic', 'obscene', 'threat',\
       'insult', 'identity_hate']
Y_train = train_data[bad_comment_cat]
This code sets up a Keras tokenizer to process text data, and extracts comment text from training and testing data for a natural language processing task.

tokenizer is a Keras tokenizer object that is used to preprocess the text data. It is initialized with several parameters:
num_words is the maximum number of words to keep, based on word frequency. Any less frequent words will be discarded.
filters is a string of characters to remove from the text data. Here, it removes punctuation marks and numbers.
lower is a Boolean parameter indicating whether to convert all text to lowercase.
split is a string indicating the separator to use when splitting the text into words. Here, it is set to split on whitespace.
X_train_raw and X_test_raw are Pandas Series objects that contain the comment text data from the training and testing sets, respectively.

bad_comment_cat is a list of labels for the six categories of bad comments that the model will try to classify: toxic, severe_toxic, obscene, threat, insult, and identity_hate.

Y_train is a Pandas DataFrame object that contains the labels for the six categories of bad comments in the training set. It is extracted from the train_data DataFrame using the bad_comment_cat labels.

In [ ]:
# preprocess data
X_train_raw = X_train_raw.apply(lambda x: preprocess(str(x)))
X_test_raw = X_test_raw.apply(lambda x: preprocess(str(x)))
# example X_train_raw.loc[126]

# tokenize comment text
tokenizer.fit_on_texts(X_train_raw)
tokenizer.fit_on_texts(X_test_raw)

X_train = pad_sequences(tokenizer.texts_to_sequences(X_train_raw),\
                        maxlen = max_len, truncating = "pre")
X_test = pad_sequences(tokenizer.texts_to_sequences(X_test_raw),\
                       maxlen = max_len, truncating = "pre")

# shuffle training data and split it into a training part and a validation part
x_train, x_val, y_train, y_val = train_test_split(X_train,Y_train,train_size=0.9, random_state=199)
print(x_train.shape," ",y_train.shape," ",x_val.shape," ",y_val.shape)

(143613, 120)   (143613, 6)   (15958, 120)   (15958, 6)


this code preprocesses the text data by applying the preprocess() function defined earlier, tokenizes the comment text using the tokenizer object, pads the tokenized sequences to a fixed length of max_len, and splits the training data into a training set and a validation set.

X_train_raw and X_test_raw are preprocessed using the preprocess() function defined earlier. The apply() method applies the function to each element of the Pandas Series object.

tokenizer.fit_on_texts() is called on both the preprocessed X_train_raw and X_test_raw Series objects. This builds the vocabulary index based on word frequency.

pad_sequences() is called on the tokenized sequences of X_train_raw and X_test_raw using the tokenizer object. This pads the sequences to a fixed length of max_len and truncates sequences that exceed that length.

train_test_split() is called on the padded training data X_train and labels Y_train to split it into a training set (x_train and y_train) and a validation set (x_val and y_val) for model training.

The dimensions of the training and validation sets are printed to confirm the correct shape.







# RNN models

In [ ]:
def get_weights(embedding_vectors,embedding_dim):
    global num_tokens,tokenizer

    # assign vectors to words using the pretrained model embedding_vectors
    embedding_weights = np.zeros((num_tokens,embedding_dim))

    # count how many words are not assigned with the pretrained model.
    # By default, vectors associated to words are zero vectors.
    misses = 0

    # the index in word_index starts with 1
    for word, i in tokenizer.word_index.items():
        vector = embedding_vectors.get(word)
        # the word_index is ordered by word frequency
        if i>=num_tokens :
            break
        elif vector is not None:
            embedding_weights[i] = vector
        else:
            if len(word)<20:
                word = Word(word)
                word = word.spellcheck()[0][0]
                vector = embedding_vectors.get(str(word))
                if vector is not None:
                    embedding_weights[i] = vector
                else:
                    misses +=1
                    #print(word)
            else:
                misses +=1
                #print(word)

    print(f"The number of missed words is {misses}")

    return embedding_weights

This function is used to create the weight matrix for the embedding layer of a neural network model. The weight matrix is used to initialize the weights of the embedding layer with pre-trained word embeddings.

The function takes two inputs:

embedding_vectors: a dictionary mapping words to their corresponding pre-trained embedding vectors
embedding_dim: the dimensionality of the embedding vectors
The function first initializes an all-zero matrix with shape (num_tokens, embedding_dim), where num_tokens is the number of tokens in the tokenizer plus one (to account for the 0 padding token). For each word in the tokenizer's word index, the function checks if the word is in the embedding_vectors dictionary. If it is, the corresponding embedding vector is assigned to the corresponding row of the weight matrix. If it is not, the function attempts to correct the spelling of the word using the Word class from the textblob library. If the corrected word is in the embedding_vectors dictionary, the corresponding embedding vector is assigned to the corresponding row of the weight matrix. If neither the original nor the corrected word is in the embedding_vectors dictionary, the corresponding row of the weight matrix is left as zeros.

The function returns the weight matrix. If any words in the tokenizer's word index were not found in the embedding_vectors dictionary, the function prints the number of missed words.







In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip

--2023-04-05 14:55:41--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 108.157.162.108, 108.157.162.35, 108.157.162.83, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|108.157.162.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1523785255 (1.4G) [application/zip]
Saving to: ‘crawl-300d-2M.vec.zip’

crawl-300d-2M.vec.z 100%[===================>]   1.42G   136MB/s    in 12s     

2023-04-05 14:55:53 (117 MB/s) - ‘crawl-300d-2M.vec.zip’ saved [1523785255/1523785255]



In [ ]:
!unzip crawl*.zip
!ls
!pwd

Archive:  crawl-300d-2M.vec.zip
  inflating: crawl-300d-2M.vec       
crawl-300d-2M.vec  crawl-300d-2M.vec.zip  drive  sample_data
/content


In [ ]:
# read the pretrained model fastText
embedding_vectors_fasttext = {}
with open("crawl-300d-2M.vec","r") as file:
    file.readline()
    for line in file:
        word , vector = line.split(maxsplit=1)
        vector = np.fromstring(vector,"float32",sep=" ")
        embedding_vectors_fasttext[word] = vector

This code reads a file named "crawl-300d-2M.vec" containing pretrained word embeddings using the fastText model. The file is read line by line and each line contains a word followed by its vector representation. The first line of the file is skipped since it contains metadata. The word and its corresponding vector are stored in a dictionary named embedding_vectors_fasttext.

In [ ]:
# assign vectors to words using the pretrained model fasttext
embedding_weights_fasttext = get_weights(embedding_vectors_fasttext,embedding_dim=300)

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

In [ ]:
!unzip glove*.zip
!ls
!pwd

In [ ]:
# read the pretrained model GloVe
embedding_vectors_glove = {}
with open("glove.6B.300d.txt","r") as file:
    for line in file:
        word , vector = line.split(maxsplit=1)
        vector = np.fromstring(vector,"float32",sep=" ")
        embedding_vectors_glove[word] = vector

In [ ]:
# assign vectors to words using the pretrained model GloVe
embedding_weights_glove = get_weights(embedding_vectors_glove,embedding_dim=300)

The code above assigns pre-trained word embeddings to words in the vocabulary using the FastText model. The get_weights function takes in two arguments, embedding_vectors and embedding_dim, where embedding_vectors is a dictionary containing the pre-trained word embeddings and embedding_dim is an integer specifying the dimensionality of the embeddings.

The function first initializes a zero matrix of shape (num_tokens, embedding_dim), where num_tokens is the vocabulary size plus one (including 0) and embedding_dim is the specified dimensionality. It then iterates over the words in the tokenizer's word_index, which is a dictionary mapping each word to its integer index, and assigns the corresponding pre-trained embedding vector if available in the embedding_vectors dictionary. If a pre-trained vector is not available for a given word, the function attempts to find a spelling correction for the word using the Word class from the textblob library and assigns the corresponding pre-trained embedding vector if available. If a pre-trained vector is still not available for a given word, the function keeps the corresponding row in the embedding_weights matrix as a zero vector.

The function returns the embedding_weights matrix, which contains the assigned pre-trained embedding vectors for the words in the vocabulary. In this case, the pre-trained embeddings are from the FastText model, and the resulting embedding_weights_fasttext matrix contains the assigned embeddings.

In [ ]:
def GRU_model_glove():
    global max_len,num_tokens,embedding_weights_glove

    inputs = layers.Input(shape=(max_len,))

    x = layers.Embedding(input_dim=num_tokens,\
                         output_dim=embedding_dim,\
                         embeddings_initializer=keras.initializers.Constant(embedding_weights_glove),\
                         trainable=True)(inputs)

    x = layers.SpatialDropout1D(0.3)(x)

    forward_layer = layers.GRU(42,return_sequences=True)
    backward_layer = layers.GRU(42,activation="relu",dropout=0.1,return_sequences=True,go_backwards=True)
    x = layers.Bidirectional(forward_layer,backward_layer=backward_layer)(x)

    x = layers.GlobalMaxPooling1D()(x)

    outputs = layers.Dense(units=6,activation='sigmoid')(x)

    model = keras.models.Model(inputs=inputs, outputs=outputs, name="GRU_model_glove")

    model.compile(optimizer=tf.optimizers.Adam(),\
                  loss=tf.losses.BinaryCrossentropy(),\
                  metrics=['AUC'])

    return model

GRU_model_glove = GRU_model_glove()
GRU_model_glove.summary()

This is a Keras model definition for a GRU model with GloVe embeddings, which can be used to classify comments into 6 categories. Here are the details of the model:

The input layer takes sequences of integers of length max_len as input.
An embedding layer is added on top of the input layer. The embedding layer uses pre-trained GloVe embeddings. The embedding layer is trainable.
A dropout layer with a dropout rate of 0.3 is added on top of the embedding layer to prevent overfitting.
A bidirectional GRU layer with 42 units is added on top of the dropout layer. The output sequences of the GRU layer are returned.
A global max pooling layer is added on top of the GRU layer. The purpose of the global max pooling layer is to extract the most important features from the GRU output sequences.
A dense layer with 6 units and a sigmoid activation function is added on top of the global max pooling layer. The dense layer outputs the predicted probabilities for each of the 6 categories.
The model is compiled with the Adam optimizer, binary crossentropy loss function, and AUC metric.
The model summary shows the number of trainable parameters for each layer and the total number of trainable parameters.




In [ ]:
history = GRU_model_glove.fit(x_train, y_train, epochs=2,\
                              batch_size=32, validation_data=(x_val,y_val))

This code is training a GRU (Gated Recurrent Unit) model using GloVe (Global Vectors for Word Representation) embeddings on a dataset with a training set x_train and corresponding labels y_train, and a validation set x_val and corresponding labels y_val.

The fit method is used to train the model on the training data. Here are the arguments used in this method:

x_train: This is the input data for the training set. It should be a numpy array or a list of numpy arrays, depending on the model's input shape.

y_train: This is the corresponding target labels for the training set. It should also be a numpy array or a list of numpy arrays, depending on the model's output shape.

epochs: This is the number of times the model will iterate over the entire training set. In this code snippet, the model will train for 2 epochs.

batch_size: This is the number of samples that will be processed in each training batch. The default batch size is 32, which is also the value used in this code snippet.

validation_data: This is the validation data used to evaluate the model's performance during training. It should also be a tuple of input data and corresponding target labels.

The fit method returns a history object, which contains information about the model's training history. This object is assigned to the variable history.

So, the code you provided is training a GRU model using GloVe embeddings on the given data for two epochs, with a batch size of 32 and using a validation set to evaluate the model's performance. The training history is then stored in the history object.






In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [ ]:
prediction = GRU_model_glove.predict(x_val)
y_pred = (prediction > 0.5)
print("Accuracy of the model : ", accuracy_score(y_pred, y_val))
print('F1-score: ', f1_score(y_pred, y_val,average='weighted'))
print("Precision Score : ",precision_score(y_val, y_pred,  average='micro'))
print("Recall Score : ",recall_score(y_val, y_pred, average='micro'))

In [ ]:
from sklearn.metrics import classification_report
target_names = ['Toxic', 'severe_toxic', 'obscene','threat','insult','identity hate']
print(classification_report(y_val, y_pred, target_names=target_names))

In [ ]:
def GRU_model_fasttext():
    global max_len,num_tokens,embedding_weights_fasttext

    inputs = layers.Input(shape=(max_len,))

    x = layers.Embedding(input_dim=num_tokens,\
                         output_dim=embedding_dim,\
                         embeddings_initializer=keras.initializers.Constant(embedding_weights_fasttext),\
                         trainable=False)(inputs)

    x = layers.SpatialDropout1D(0.3)(x)

    forward_layer = layers.GRU(64,return_sequences=True)
    backward_layer = layers.GRU(64,activation="relu",dropout=0.3,return_sequences=True,go_backwards=True)
    x = layers.Bidirectional(forward_layer,backward_layer=backward_layer)(x)

    avg_pool = layers.GlobalAveragePooling1D()(x)
    max_pool = layers.GlobalMaxPooling1D()(x)
    x = layers.concatenate([avg_pool,max_pool])

    outputs = layers.Dense(units=6,activation='sigmoid')(x)

    model = keras.models.Model(inputs=inputs, outputs=outputs, name="GRU_model")

    model.compile(optimizer=tf.optimizers.Adam(),\
                  loss=tf.losses.BinaryCrossentropy(),\
                  metrics=['AUC'])

    return model

GRU_model_fasttext = GRU_model_fasttext()
GRU_model_fasttext.summary()


In [ ]:
history1 = GRU_model_fasttext.fit(x_train, y_train, \
                                 epochs=2, batch_size=32,\
                                 validation_data=(x_val,y_val))

In [ ]:
prediction2 = GRU_model_fasttext.predict(x_val)
y_pred = (prediction2 > 0.5)
print("Accuracy of the model : ", accuracy_score(y_pred, y_val))
print('F1-score: ', f1_score(y_pred, y_val,average='weighted'))
print("Precision Score : ",precision_score(y_val, y_pred,  average='micro'))
print("Recall Score : ",recall_score(y_val, y_pred, average='micro'))

In [ ]:
from sklearn.metrics import classification_report
target_names = ['Toxic', 'severe_toxic', 'obscene','threat','insult','identity hate']
print(classification_report(y_val, y_pred, target_names=target_names))

# Ensemble

In [ ]:
model_nums = 2
size1 = x_train.shape[0]

y_train_pred = np.zeros((model_nums,size1,6),dtype="float32")
y_train_pred[0] = GRU_model_fasttext.predict(x_train)
y_train_pred[1] = GRU_model_glove.predict(x_train)

size2 = X_test.shape[0]
y_test_pred = np.zeros((model_nums,size2,6),dtype="float32")
y_test_pred[0] = GRU_model_fasttext.predict(X_test)
y_test_pred[1] = GRU_model_glove.predict(X_test)

y_pred = np.zeros((size2,6),dtype="float32")

for i in range(6):
    lg = LogisticRegression()
    temp = np.zeros((size1,model_nums),dtype="float32")
    for j in range(model_nums):
        temp[:,j] = y_train_pred[j,:,i]
    lg.fit(temp,y_train[bad_comment_cat[i]])

    temp = np.zeros((size2,model_nums),dtype="float32")
    for j in range(model_nums):
        temp[:,j] = y_test_pred[j,:,i]
    y_pred[:,i] = lg.predict_proba(temp)[:,1]

# Submission

In [ ]:
submission_result[bad_comment_cat] = y_pred
submission_result.to_csv("submission.csv",index=False)

In [ ]:
submission = pd.read_csv('submission.csv')

In [ ]:
submission.head(10)